In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv('../Data/Split_highway.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df = df.drop(['version', 'id_2'], axis=1)

In [ ]:
df['tags'].value_counts()

In [ ]:
groups = df.groupby('fid')

In [ ]:
tile_value_count_df = df['fid'].value_counts().sort_values().to_frame().reset_index().rename(columns={'index': 'tile_id','fid':'count'})

In [ ]:
tile_value_count_df[tile_value_count_df['count']==1]['tile_id']

In [ ]:
temp_tile_count_one = df[df['fid'].isin(tile_value_count_df[tile_value_count_df['count']==1]['tile_id'])]

In [ ]:
temp_tile_count_one

In [ ]:
temp_tile_count_one.columns
for col in temp_tile_count_one.columns:
    print(f"{col}: {temp_tile_count_one[col].isnull().values.all()}, \
    {temp_tile_count_one[col].isnull().mean() * 100}")


In [ ]:
df

In [ ]:
df[df['fid']==10725]['bicycle'].value_counts()

In [ ]:
df['fid'].value_counts()

In [ ]:
for col in df.columns:
    print(col)
    print(df[col].value_counts(dropna=False,normalize=True))
    print("\n\n\n")
# df['fid'].value_counts()

## ROAD VECTORS TILED 

In [3]:
df = pd.read_csv('../Data/Road_Vectors_Tiled.csv')
df = df.rename(columns={"id": "Tile_ID"})

In [4]:
df.head()
print(df['RTTYP'].value_counts(dropna=False,normalize=False))


M      25271
NaN     1193
I        423
S        357
U        286
O         11
Name: RTTYP, dtype: int64


In [5]:
import random

sampleList = ['M', 'I', 'S', 'U', 'O']

randomList = random.choices(
sampleList, weights=(1, 0.01673, 0.014126, 0.011317, 0.000435))
list_of_randoms = []
for x in range(1193):
    randomList = random.choices(
    sampleList, weights=(1, 0.01673, 0.014126, 0.011317, 0.000435))
    list_of_randoms.append(randomList[0])
    
print(len(list_of_randoms))

1193


In [6]:
def mapper(x):
    if x in ['M','I','S','U','O']:
        return x
    else :
        return list_of_randoms.pop()
df['RTTYP']=df['RTTYP'].apply(mapper)
df['RTTYP'].value_counts(dropna=False)

M    26424
I      439
S      368
U      299
O       11
Name: RTTYP, dtype: int64

In [ ]:
# count_roadtype_per_tile = df.Tile_ID.value_counts().reset_index().\
#         rename(columns={'Tile_ID':'count','index':'Tile_ID'})

# count_roadtype_per_tile[count_roadtype_per_tile['count']>1]

In [7]:
def get_frequent(series):
    return series.value_counts(ascending=False).to_frame().index[0]

tile_to_roadtype_map = df[['Tile_ID','RTTYP']].dropna().groupby('Tile_ID').agg({'RTTYP':[get_frequent]})
tile_to_roadtype_map = tile_to_roadtype_map['RTTYP']['get_frequent'].reset_index()

In [8]:
# df['Tile_ID'].value_counts()
df[df['Tile_ID']==11109]['RTTYP'].value_counts()

M    23
U     1
I     1
Name: RTTYP, dtype: int64

In [9]:
tile_to_roadtype_map.loc[tile_to_roadtype_map['Tile_ID']==4030]['get_frequent'].values[0]
def mapper_val(x):
    temp = tile_to_roadtype_map.loc[tile_to_roadtype_map['Tile_ID']==x]['get_frequent'].values
    if len(temp)>0:
        return temp[0]
    return None
df['RTTYP']=df['Tile_ID'].apply(mapper_val)

In [10]:
# Verifying
df[df['Tile_ID']==10777]['RTTYP'].value_counts()
# df['Tile_ID'].value_counts()

M    17
Name: RTTYP, dtype: int64

In [14]:
df['RTTYP'].value_counts()

M    27096
I      220
S      130
U       93
O        2
Name: RTTYP, dtype: int64

In [ ]:
df_orig = pd.read_csv('../Data/Tiles_expanded.csv')

In [ ]:
df_orig.columns

In [ ]:
joined_df = df_orig.join(df,on='Tile_ID',how='inner',lsuffix='_left',rsuffix='_right')

In [ ]:
joined_df.columns

In [ ]:
joined_df = joined_df.drop(['Tile_ID_left','wkt_geom','fid',
                'LINEARID','FULLNAME','Tile_ID_right','left','top','right','bottom','MTFCC'],axis=1)

In [ ]:
joined_df['RTTYP'].value_counts(dropna=False)

In [ ]:
joined_df

In [ ]:
!pwd

In [ ]:
joined_df.to_csv('/Users/abajaj/Documents/UC_Berkeley/GIT/210_Capstone_Aditya_Arisa_Noriel/Data/Tiles_expanded_road_vector.csv')